In [125]:
import Pkg; Pkg.status()
include("Arthur.jl")
using RigidBodyDynamics
using StaticArrays
using RobotDynamics
using Rotations
using LinearAlgebra
using ForwardDiff, FiniteDiff
using Altro
using TrajectoryOptimization

      Status `~/Documents/16681_MRSD_Proj_1/Arthur/src/Project.toml` (empty project)


In [133]:
model = Arthur()
print(RobotDynamics.dynamics(model, zeros(32), 0.1*ones(7)))
n,m = size(model)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1200217459844497, 0.1583279359589722, -2.0445209801704287, -3.0733176145180745, 4.5216697765519385, 9.570781093452561, 146.26706470438666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.9165545198739204, -7.137439051109092e-14, -2.363560580935879e-15, -4.683167267134137e-14, 6.656885032226565, -148.86416189847452, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

(32, 7)

In [127]:
N = 61
tf = 3.
dt = tf/(N-1)

0.05

In [128]:
# TODO:
# Uref could be just U equilibrium (just gravity compensation),
# Xref could be the desired joint and cartesian positions and velocities, and then 0 for the forces
Uref = [fill(0.0, m) for k = 1:N-1]
Xref = [fill(0.0, n) for k = 1:N]
traj = RobotDynamics.Traj(Xref, Uref, [dt for k=1:N], cumsum(dtref) .- dtref[1]);

In [129]:
# Set up
# TODO: Tune weights
Q = 1.0e-2*Diagonal(@SVector ones(n))
Qf = 100.0*Diagonal(@SVector ones(n))
R = 1.0e-1*Diagonal(@SVector ones(m))
obj = TrajectoryOptimization.TrackingObjective(Q, R, traj, Qf=Qf)
# obj = LQRObjective(Q,R,Qf,xf,N);

In [130]:
# Create Empty ConstraintList
conSet = ConstraintList(n,m,N)

# Control Bounds based on Robot Specs (Joint torque limits)
u_bnd = [39.0, 39.0, 39.0, 39.0, 9.0, 9.0, 9.0]
control_bnd = BoundConstraint(n,m, u_min=-u_bnd, u_max=u_bnd)
add_constraint!(conSet, control_bnd, 1:N-1)

# State Bounds based on Robot Specs (Joint velocity and speed limits)
x_bnd = zeros(n)
x_bnd[1:7] = [Inf, deg2rad(128.9), Inf, deg2rad(147.8), Inf, deg2rad(120.3), Inf] # rad
x_bnd[8:14] = [1.39, 1.39, 1.39, 1.39, 1.22, 1.22, 1.22] # rad/sec
x_bnd[15:end] = [Inf for k=1:(n-14)] # Constraints on force elsewhere
state_bnd = BoundConstraint(n,m, x_min=-x_bnd, x_max=x_bnd)
add_constraint!(conSet, state_bnd, 1:N)

# Cartesian Velocity Bound
ẋ_max = 0.0005 # m/s
vel_bnd = NormConstraint(n, m, ẋ_max, Inequality(), 21:23)
add_constraint!(conSet, vel_bnd, 1:N)

# Force Bound (Fx Fy Fz)
F_max = 20 # Newtons
F_bnd = NormConstraint(n, m, F_max, Inequality(), 27:29)
add_constraint!(conSet, F_bnd, 1:N)

# Other possible constraints:
# Cartesian angular velocity bounds?
# External moment bounds?

# Goal Constraint - only if you want the final state to be the desired state
# goal = GoalConstraint(xf)
# add_constraint!(conSet, goal, N)

In [131]:
prob = Problem(model, obj, Xref[end], tf, x0=Xref[1], constraints=conSet, N=N, U0=Uref, X0=Xref)

Problem{RK3, Float64}(Arthur{Any}(Spanning tree:
Vertex: world (root)
  Vertex: Shoulder_Link, Edge: Actuator1
    Vertex: HalfArm1_Link, Edge: Actuator2
      Vertex: HalfArm2_Link, Edge: Actuator3
        Vertex: ForeArm_Link, Edge: Actuator4
          Vertex: SphericalWrist1_Link, Edge: Actuator5
            Vertex: SphericalWrist2_Link, Edge: Actuator6
              Vertex: Bracelet_Link, Edge: Actuator7
No non-tree joints.), Objective, ConstraintList(32, 7, TrajectoryOptimization.AbstractConstraint[BoundConstraint{14, 39, Float64}(32, 7, [Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf  …  Inf, Inf, Inf, 39.0, 39.0, 39.0, 39.0, 9.0, 9.0, 9.0], [-Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf  …  -Inf, -Inf, -Inf, -39.0, -39.0, -39.0, -39.0, -9.0, -9.0, -9.0], [449, 464, 479, 494, 509, 524, 539], [456, 471, 486, 501, 516, 531, 546], [33, 34, 35, 36, 37, 38, 39, 72, 73, 74, 75, 76, 77, 78]), BoundConstraint{20, 39, Float64}(32, 7, [Inf, 2.2497294058206907, Inf, 2.5795966

In [132]:
using Altro
opts = SolverOptions(
    cost_tolerance_intermediate=1e-2,
    penalty_scaling=10.,
    penalty_initial=1.0
)

altro = ALTROSolver(prob, opts)
set_options!(altro, show_summary=true)
solve!(altro);

LoadError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{RobotDynamics.var"#fd_aug#16"{RK3, Arthur{Any}, Float64, Float64, SVector{7, Int64}, SVector{32, Int64}}, Float64}, Float64, 39})
[0mClosest candidates are:
[0m  (::Type{T})(::Real, [91m::RoundingMode[39m) where T<:AbstractFloat at rounding.jl:200
[0m  (::Type{T})(::T) where T<:Number at boot.jl:760
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{AbstractChar, Number} at char.jl:50
[0m  ...